<a href="https://colab.research.google.com/github/kiaerii/mash/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests beautifulsoup4 pandas scikit-learn pymorphy3 sentence-transformers

In [ ]:
import datetime as dt
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import Counter
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
import pymorphy3
from sentence_transformers import SentenceTransformer

BASE = "https://lenta.ru"

def parse_full_text(url: str) -> str:
    """Парсинг полного текста новости по ссылке."""
    try:
        # Добавляем заголовки для имитации браузера
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        r.encoding = 'utf-8'  # Явно указываем кодировку
        soup = BeautifulSoup(r.content, "html.parser")

        # Более гибкий поиск текстовых блоков
        # Ищем по различным классам, которые могут содержать текст статьи
        text_blocks = []

        # 1. Ищем параграфы с классами, содержащими 'body'
        text_blocks = soup.find_all("p", class_=lambda x: x and ("body" in str(x).lower() or "article" in str(x).lower()))

        # 2. Если не нашли, ищем по тегам внутри article
        if not text_blocks:
            article = soup.find("article")
            if article:
                text_blocks = article.find_all("p")

        # 3. Последний вариант - все параграфы
        if not text_blocks:
            text_blocks = soup.find_all("p")

        # Фильтруем слишком короткие блоки
        filtered_blocks = []
        for block in text_blocks:
            text = block.get_text(strip=True)
            if len(text) > 20:  # Игнорируем очень короткие блоки
                filtered_blocks.append(text)

        # Объединяем текст
        text = " ".join(filtered_blocks)
        return text[:1500]  # Ограничиваем размер для экономии памяти
    except Exception as e:
        print(f"Ошибка при парсинге текста {url}: {e}")
        return ""

def parse_news(news_count=50, date_str: str | None = None) -> pd.DataFrame:
    """
    Сбор новостей с главной страницы или по конкретной дате.
    news_count: сколько новостей собрать
    date_str: строка "YYYY-MM-DD" или None (тогда берём главную)
    """
    # Добавляем заголовки для запросов
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    if date_str:
        try:
            y, m, d = map(int, date_str.split("-"))
            url = f"{BASE}/news/{y:04d}/{m:02d}/{d:02d}/"
        except:
            print("Неверный формат даты. Используется главная страница.")
            url = BASE + "/"
    else:
        url = BASE + "/"

    print("Парсим URL:", url)

    try:
        r = requests.get(url, headers=headers, timeout=10)
        r.raise_for_status()
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.content, "html.parser")
        news = []

        items = []

        if date_str and "/news/" in url:
            # Для страницы с конкретной датой
            prefix = f"/news/{y:04d}/{m:02d}/{d:02d}/"
            all_links = soup.find_all("a", href=True)
            items = [a for a in all_links if a["href"].startswith(prefix)]
            print(f"Найдено ссылок на новости за {date_str}: {len(items)}")
        else:
            # Для главной страницы
            # Ищем по различным классам
            items = soup.find_all("a", class_=lambda x: x and any(
                word in str(x).lower() for word in ["card", "news", "item", "topnews", "banner", "tile"]
            ))

            # Если не нашли по классам, ищем все ссылки с /news/
            if len(items) < news_count:
                all_links = soup.find_all("a", href=True)
                items = [a for a in all_links if "/news/" in a.get("href", "")]

        print(f"Всего найдено потенциальных новостей: {len(items)}")

        for idx, a in enumerate(items):
            if len(news) >= news_count:
                break

            try:
                # Заголовок
                title = ""
                title_elem = a.find(["h3", "span", "div"])
                if title_elem:
                    title = title_elem.get_text(strip=True)
                else:
                    title = a.get_text(strip=True)

                # Пропускаем слишком короткие заголовки
                if len(title) < 10:
                    continue

                # Ссылка
                link = a.get("href")
                if not link:
                    continue

                if link and not link.startswith("http"):
                    if link.startswith("//"):
                        link = "https:" + link
                    else:
                        link = BASE + link

                # Полный текст
                print(f"Обработка новости {len(news)+1}/{news_count}: {title[:50]}...")
                full_text = parse_full_text(link)

                news.append({
                    "date": date_str or dt.date.today().isoformat(),
                    "title": title,
                    "link": link,
                    "full_text": full_text,
                    "text_length": len(full_text)
                })

            except Exception as e:
                print(f"Ошибка при обработке новости {idx}: {e}")
                continue

        df = pd.DataFrame(news)
        print(f"Собрано новостей: {len(df)}")
        return df

    except Exception as e:
        print(f"Ошибка при парсинге страницы: {e}")
        # Возвращаем пустой DataFrame
        return pd.DataFrame(columns=["date", "title", "link", "full_text", "text_length"])

In [ ]:
# Настройки
NEWS_COUNT = 50
DATE_STR = None  # Можно указать дату, например "2024-03-15"

print("=" * 70)
print(" СБОР НОВОСТЕЙ С LENTA.RU")
print("=" * 70)

# Собираем новости
df = parse_news(news_count=NEWS_COUNT, date_str=DATE_STR)

print("\n" + "=" * 70)
print(" РЕЗУЛЬТАТЫ СБОРА ДАННЫХ")
print("=" * 70)

# Показываем статистику
print(f"\n Собрано новостей: {len(df)}")
print(f" Период: {df['date'].iloc[0]}")
print(f" Средняя длина текста: {df['text_length'].mean():.0f} символов")

# Создаем красивую таблицу для отображения
from IPython.display import display, HTML

print("\n" + "=" * 70)
print(" ТАБЛИЦА НОВОСТЕЙ (первые 10 записей)")
print("=" * 70)

# Создаем стилизованную таблицу
styled_df = df.head(10).copy()
styled_df['№'] = range(1, len(styled_df) + 1)
styled_df['Заголовок'] = styled_df['title'].apply(lambda x: x[:80] + '...' if len(x) > 80 else x)
styled_df['Длина текста'] = styled_df['text_length']
styled_df['Дата'] = styled_df['date']
styled_df['Ссылка'] = styled_df['link'].apply(lambda x: f'<a href="{x}" target="_blank">Открыть</a>')

# Отображаем таблицу
display_df = styled_df[['№', 'Дата', 'Заголовок', 'Длина текста', 'Ссылка']]

# Создаем HTML таблицу с стилями
html = """
<style>
.news-table {
    width: 100%;
    border-collapse: collapse;
    font-family: Arial, sans-serif;
    margin: 20px 0;
}
.news-table th {
    background-color: #4CAF50;
    color: white;
    padding: 12px;
    text-align: left;
    border: 1px solid #ddd;
}
.news-table td {
    padding: 10px;
    border: 1px solid #ddd;
}
.news-table tr:nth-child(even) {
    background-color: #f2f2f2;
}
.news-table tr:hover {
    background-color: #ddd;
}
.news-link {
    color: #0066cc;
    text-decoration: none;
}
.news-link:hover {
    text-decoration: underline;
}
</style>
"""

html += "<table class='news-table'>"
html += "<tr><th>№</th><th>Дата</th><th>Заголовок</th><th>Длина текста</th><th>Ссылка</th></tr>"

for _, row in display_df.iterrows():
    html += f"""
    <tr>
        <td>{row['№']}</td>
        <td>{row['Дата']}</td>
        <td>{row['Заголовок']}</td>
        <td>{row['Длина текста']}</td>
        <td>{row['Ссылка']}</td>
    </tr>
    """

html += "</table>"

# Добавляем статистику под таблицей
html += f"""
<div style="margin-top: 20px; padding: 10px; background-color: #f8f9fa; border-left: 4px solid #4CAF50;">
    <h4> Статистика:</h4>
    <p>• Всего новостей: <strong>{len(df)}</strong></p>
    <p>• Средняя длина текста: <strong>{df['text_length'].mean():.0f}</strong> символов</p>
    <p>• Минимальная длина: <strong>{df['text_length'].min()}</strong> символов</p>
    <p>• Максимальная длина: <strong>{df['text_length'].max()}</strong> символов</p>
</div>
"""

# Выводим HTML
display(HTML(html))

# Также показываем стандартный DataFrame для полноты информации
print("\n" + "=" * 70)
print(" ПОЛНЫЙ DATAFRAME С НОВОСТЯМИ")
print("=" * 70)
print(f"\nРазмер DataFrame: {df.shape}")
print(f"Колонки: {list(df.columns)}")

# Показываем первые 5 строк в обычном виде
print("\nПервые 5 строк DataFrame:")
pd.set_option('display.max_colwidth', 80)  # Увеличиваем ширину колонок
print(df[['date', 'title', 'text_length']].head())
pd.reset_option('display.max_colwidth')

 СБОР НОВОСТЕЙ С LENTA.RU
Парсим URL: https://lenta.ru/
Всего найдено потенциальных новостей: 158
Обработка новости 1/50: В Киеве раздались взрывы01:41...
Обработка новости 2/50: В Европе подвели итоги противостояния с Россией01:...
Обработка новости 3/50: Жители российского региона сообщили о взрывах и вс...
Обработка новости 4/50: Зеленский прокомментировал переговоры Уиткоффа и П...
Обработка новости 5/50: Германия передвинула истребители ближе к границам ...
Обработка новости 6/50: Трамп ужесточил правила для мигрантов01:05...
Обработка новости 7/50: Самая красивая женщина в мире показала фото в крош...
Обработка новости 8/50: Названы тревожные причины отсутствия секса в паре0...
Обработка новости 9/50: В США предложили запретить Nvidia экспортировать ч...
Обработка новости 10/50: Мерц высказался об опасениях Бельгии по поводу акт...
Обработка новости 11/50: Дмитриев обратился к Мерцу00:29...
Обработка новости 12/50: Киркоров подарил Собчак сумку-ежа почти за 100 тыс...
Обработка н

№,Дата,Заголовок,Длина текста,Ссылка
1,2025-12-04,В Киеве раздались взрывы01:41,557,Открыть
2,2025-12-04,В Европе подвели итоги противостояния с Россией01:41,1093,Открыть
3,2025-12-04,Жители российского региона сообщили о взрывах и вспышках01:39,528,Открыть
4,2025-12-04,Зеленский прокомментировал переговоры Уиткоффа и Путина01:18,891,Открыть
5,2025-12-04,Германия передвинула истребители ближе к границам России01:17,685,Открыть
6,2025-12-04,Трамп ужесточил правила для мигрантов01:05,1151,Открыть
7,2025-12-04,Самая красивая женщина в мире показала фото в крошечном бюстгальтере01:02,870,Открыть
8,2025-12-04,Названы тревожные причины отсутствия секса в паре01:02,1444,Открыть
9,2025-12-04,В США предложили запретить Nvidia экспортировать чипы в Россию00:44,870,Открыть
10,2025-12-04,Мерц высказался об опасениях Бельгии по поводу активов России00:44,1329,Открыть



 ПОЛНЫЙ DATAFRAME С НОВОСТЯМИ

Размер DataFrame: (50, 5)
Колонки: ['date', 'title', 'link', 'full_text', 'text_length']

Первые 5 строк DataFrame:
         date                                                          title  \
0  2025-12-04                                  В Киеве раздались взрывы01:41   
1  2025-12-04           В Европе подвели итоги противостояния с Россией01:41   
2  2025-12-04  Жители российского региона сообщили о взрывах и вспышках01:39   
3  2025-12-04   Зеленский прокомментировал переговоры Уиткоффа и Путина01:18   
4  2025-12-04  Германия передвинула истребители ближе к границам России01:17   

   text_length  
0          557  
1         1093  
2          528  
3          891  
4          685  


In [ ]:
def clean_text(text: str) -> str:
    """
    Очистка текста:
    - в нижний регистр
    - удаление знаков препинания и прочих символов, кроме русских/латинских букв и пробелов
    - сжатие повторяющихся пробелов
    - удаление цифр
    """
    if not isinstance(text, str):
        return ""

    text = text.lower()
    text = re.sub(r'[^a-zа-яё\s]', ' ', text, flags=re.IGNORECASE)
    text = re.sub(r'\d+', ' ', text)  # Удаляем цифры
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def build_frequency_dictionary(token_lists) -> Counter:
    """Построение частотного словаря по списку списков токенов."""
    all_tokens = []
    for tokens in token_lists:
        if isinstance(tokens, list):
            # Фильтруем очень короткие слова
            filtered_tokens = [token for token in tokens if len(token) > 2]
            all_tokens.extend(filtered_tokens)

    freq = Counter(all_tokens)
    return freq

# Объединяем заголовок и текст новости
df["full_text"] = df["full_text"].fillna("")
df["title"] = df["title"].fillna("")
df["text"] = df["title"] + " " + df["full_text"]

# Очистка текста
print("Очистка текста...")
df["clean_text"] = df["text"].astype(str).apply(clean_text)

# Токенизация (по пробелу)
print("Токенизация...")
df["tokens"] = df["clean_text"].apply(lambda x: x.split())

# Частотный словарь
print("Построение частотного словаря...")
freq_dict = build_frequency_dictionary(df["tokens"])
freq_df = pd.DataFrame(freq_dict.most_common(), columns=["word", "freq"])

print("\nТоп-20 самых частотных слов:")
print(freq_df.head(20))

print(f"\nВсего уникальных слов: {len(freq_dict)}")
print(f"Всего слов в текстах: {sum(freq_dict.values())}")

# Bag of Words (BoW)
print("\nСоздание Bag of Words...")
vectorizer = CountVectorizer(
    tokenizer=lambda x: x.split(),  # мы уже токенизируем через пробел
    preprocessor=lambda x: x,       # текст уже очищен
    max_features=1000,              # Ограничиваем размер словаря для наглядности
    min_df=2                        # Слова, встречающиеся хотя бы в 2 документах
)

X_bow = vectorizer.fit_transform(df["clean_text"])
bow_df = pd.DataFrame(
    X_bow.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print("Форма матрицы Bag of Words:", X_bow.shape)
print(f"Размер словаря BoW: {len(vectorizer.get_feature_names_out())}")

# Показываем пример BoW представления
print("\nПример BoW представления для первой новости:")
first_row = bow_df.iloc[0]
nonzero_cols = first_row[first_row > 0]
print(f"Ненулевых элементов: {len(nonzero_cols)}")
print("Слова с частотой > 0:")
for word, freq in nonzero_cols.head(10).items():
    print(f"  {word}: {int(freq)}")

Очистка текста...
Токенизация...
Построение частотного словаря...

Топ-20 самых частотных слов:
          word  freq
0          что    87
1      декабря    39
2        ранее    30
3       словам    28
4          как    26
5          для    25
6          она    20
7         этом    19
8          его    19
9          это    18
10      россии    16
11         над    15
12         сша    14
13  президента    13
14       также    13
15       после    13
16       время    12
17        года    12
18   президент    12
19         том    11

Всего уникальных слов: 2992
Всего слов в текстах: 4777

Создание Bag of Words...
Форма матрицы Bag of Words: (50, 559)
Размер словаря BoW: 559

Пример BoW представления для первой новости:
Ненулевых элементов: 34
Слова с частотой > 0:
  были: 1
  в: 7
  вечером: 1
  взрывов: 1
  взрывы: 4
  говорится: 1
  города: 1
  данным: 2
  декабря: 2
  другие: 1


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
import pymorphy3

morph = pymorphy3.MorphAnalyzer()

def lemmatize_tokens(tokens, morph_analyzer) -> list[str]:
    """Лемматизация списка токенов с помощью pymorphy3."""
    lemmas = []
    for token in tokens:
        if len(token) > 2:  # Пропускаем очень короткие слова
            try:
                parsed = morph_analyzer.parse(token)[0]
                lemmas.append(parsed.normal_form)
            except:
                lemmas.append(token)  # Если не удалось лемматизировать, оставляем как есть
    return lemmas

print("Лемматизация токенов...")
df["lemmas"] = df["tokens"].apply(lambda tokens: lemmatize_tokens(tokens, morph))
df["lemmas_text"] = df["lemmas"].apply(lambda tokens: " ".join(tokens))

print("\nПримеры обработки текста:")
print("=" * 60)

for i in range(min(3, len(df))):
    print(f"\nНовость {i+1}:")
    print(f"Оригинальный заголовок: {df['title'].iloc[i][:100]}...")
    print(f"Очищенный текст (первые 100 символов): {df['clean_text'].iloc[i][:100]}...")
    print(f"Лемматизированный текст (первые 100 символов): {df['lemmas_text'].iloc[i][:100]}...")
    print(f"Количество токенов: {len(df['tokens'].iloc[i])}")
    print(f"Количество лемм: {len(df['lemmas'].iloc[i])}")
    print("-" * 40)

# Частотный словарь для лемматизированных текстов
print("\nСоздание частотного словаря для лемм...")
lemmas_freq_dict = build_frequency_dictionary(df["lemmas"])
lemmas_freq_df = pd.DataFrame(lemmas_freq_dict.most_common(), columns=["lemma", "freq"])

print("\nТоп-20 самых частотных лемм:")
print(lemmas_freq_df.head(20))

Лемматизация токенов...

Примеры обработки текста:

Новость 1:
Оригинальный заголовок: В Киеве раздались взрывы01:41...
Очищенный текст (первые 100 символов): в киеве раздались взрывы взрывы раздались в украинской столице на фоне воздушной тревоги об этом дек...
Лемматизированный текст (первые 100 символов): киев раздаться взрыв взрыв раздаться украинский столица фон воздушный тревога это декабрь вtelegramс...
Количество токенов: 67
Количество лемм: 54
----------------------------------------

Новость 2:
Оригинальный заголовок: В Европе подвели итоги противостояния с Россией01:41...
Очищенный текст (первые 100 символов): в европе подвели итоги противостояния с россией противостояние с россией в рамках украинского конфли...
Лемматизированный текст (первые 100 символов): европа подвести итог противостояние россия противостояние россия рамка украинский конфликт привести ...
Количество токенов: 143
Количество лемм: 112
----------------------------------------

Новость 3:
Оригинальный загол

In [ ]:
# Инициализация модели для эмбеддингов
print("Загрузка модели для эмбеддингов...")
try:
    # Используем модель, которая поддерживает русский язык
    model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
    print("Модель успешно загружена!")
except Exception as e:
    print(f"Ошибка при загрузке модели: {e}")
    print("Попытка загрузить альтернативную модель...")
    try:
        model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
        print("Альтернативная модель загружена!")
    except:
        print("Не удалось загрузить модель. Создаем случайные эмбеддинги для демонстрации.")
        model = None

# Получаем эмбеддинги
print("\nПолучение эмбеддингов для лемматизированных текстов...")

if model is not None:
    # Используем реальную модель
    embeddings = model.encode(
        df["lemmas_text"].tolist(),
        convert_to_numpy=True,
        show_progress_bar=True,
        batch_size=16,
        normalize_embeddings=True
    )
else:
    # Создаем случайные эмбеддинги для демонстрации
    print("Создание случайных эмбеддингов...")
    embeddings = np.random.randn(len(df), 384)  # 384 - стандартная размерность для MiniLM
    # Нормализуем
    norms = np.linalg.norm(embeddings, axis=1, keepdims=True)
    embeddings = embeddings / norms

print(f"Форма матрицы эмбеддингов: {embeddings.shape}")
print(f"Размерность одного эмбеддинга: {embeddings.shape[1]}")

# Сохраняем эмбеддинги в DataFrame
# Для экономии памяти сохраняем только первые 10 измерений в основном DataFrame
df["embedding_sample"] = list(embeddings[:, :10])

# Создаем отдельный DataFrame для полных эмбеддингов
embedding_cols = [f"emb_{i}" for i in range(embeddings.shape[1])]
embeddings_df = pd.DataFrame(embeddings, columns=embedding_cols)

print("\nПервые 3 эмбеддинга (первые 5 измерений):")
for i in range(min(3, len(embeddings))):
    print(f"Новость {i+1}: {embeddings[i, :5].round(4)}...")

# Анализ эмбеддингов
print("\nАнализ эмбеддингов:")
print(f"Минимальное значение: {embeddings.min():.4f}")
print(f"Среднее значение: {embeddings.mean():.4f}")
print(f"Максимальное значение: {embeddings.max():.4f}")
print(f"Стандартное отклонение: {embeddings.std():.4f}")

# Пример вычисления косинусного сходства между первыми двумя новостями
if len(embeddings) >= 2:
    from sklearn.metrics.pairwise import cosine_similarity
    similarity = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
    print(f"\nКосинусное сходство между новостью 1 и 2: {similarity:.4f}")

    # Сравнение заголовков
    print(f"Заголовок 1: {df['title'].iloc[0][:50]}...")
    print(f"Заголовок 2: {df['title'].iloc[1][:50]}...")

Загрузка модели для эмбеддингов...
Модель успешно загружена!

Получение эмбеддингов для лемматизированных текстов...


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Форма матрицы эмбеддингов: (50, 384)
Размерность одного эмбеддинга: 384

Первые 3 эмбеддинга (первые 5 измерений):
Новость 1: [-0.0102  0.0649 -0.0072 -0.0299  0.0568]...
Новость 2: [-0.0289  0.0426 -0.0898  0.0443  0.0571]...
Новость 3: [ 0.0051  0.0253 -0.0057 -0.0201  0.0014]...

Анализ эмбеддингов:
Минимальное значение: -0.2041
Среднее значение: 0.0001
Максимальное значение: 0.3694
Стандартное отклонение: 0.0510

Косинусное сходство между новостью 1 и 2: 0.4819
Заголовок 1: В Киеве раздались взрывы01:41...
Заголовок 2: В Европе подвели итоги противостояния с Россией01:...


In [ ]:
import os

# Сохраняем все данные
print("Сохранение результатов...")

# 1. Основной DataFrame с текстами и признаками
df.to_csv("news_with_features.csv", index=False, encoding="utf-8")
print("✓ news_with_features.csv - основной DataFrame с текстами и признаками")

# 2. Частотный словарь исходных слов
freq_df.to_csv("freq_dict_original.csv", index=False, encoding="utf-8")
print("✓ freq_dict_original.csv - частотный словарь исходных слов")

# 3. Частотный словарь лемм
lemmas_freq_df.to_csv("freq_dict_lemmas.csv", index=False, encoding="utf-8")
print("✓ freq_dict_lemmas.csv - частотный словарь лемм")

# 4. Bag of Words матрица (первые 1000 признаков)
bow_df.to_csv("bow_matrix.csv", index=False, encoding="utf-8")
print("✓ bow_matrix.csv - матрица Bag of Words")

# 5. Полные эмбеддинги
embeddings_df.to_csv("embeddings_full.csv", index=False, encoding="utf-8")
print("✓ embeddings_full.csv - полные векторные представления")

# 6. Сводная статистика
stats = {
    "total_news": len(df),
    "avg_text_length": df["text"].str.len().mean(),
    "unique_words_original": len(freq_dict),
    "unique_words_lemmas": len(lemmas_freq_dict),
    "bow_vocabulary_size": bow_df.shape[1],
    "embedding_dimension": embeddings.shape[1],
    "avg_tokens_per_news": df["tokens"].apply(len).mean(),
    "avg_lemmas_per_news": df["lemmas"].apply(len).mean()
}

stats_df = pd.DataFrame([stats])
stats_df.to_csv("processing_stats.csv", index=False, encoding="utf-8")
print("✓ processing_stats.csv - статистика обработки")

# Создаем README файл с описанием
readme_content = """# Результаты обработки новостей

## Файлы:

1. **news_with_features.csv** - основной файл с новостями и признаками:
   - date: дата новости
   - title: заголовок
   - link: ссылка
   - full_text: полный текст
   - text_length: длина текста
   - text: объединенный текст (заголовок + полный текст)
   - clean_text: очищенный текст
   - tokens: токены (список)
   - lemmas: леммы (список)
   - lemmas_text: лемматизированный текст (строка)
   - embedding_sample: первые 10 измерений эмбеддинга

2. **freq_dict_original.csv** - частотный словарь исходных слов

3. **freq_dict_lemmas.csv** - частотный словарь лемм

4. **bow_matrix.csv** - матрица Bag of Words (1000 самых частых слов)

5. **embeddings_full.csv** - полные векторные представления (эмбеддинги)

6. **processing_stats.csv** - статистика обработки

## Обработка включала:
1. Парсинг 50 новостей с Lenta.ru
2. Очистку текста (удаление пунктуации, цифр, приведение к нижнему регистру)
3. Токенизацию
4. Создание частотного словаря
5. Кодирование в Bag of Words
6. Лемматизацию с помощью pymorphy3
7. Векторизацию с помощью Sentence-Transformers
"""

with open("README.txt", "w", encoding="utf-8") as f:
    f.write(readme_content)
print("✓ README.txt - описание файлов и процесса обработки")

print("\n" + "="*60)
print("ВСЕ ФАЙЛЫ УСПЕШНО СОХРАНЕНЫ!")
print("="*60)

# Показываем размеры файлов
print("\nРазмеры файлов:")
for filename in ["news_with_features.csv", "freq_dict_original.csv",
                 "freq_dict_lemmas.csv", "bow_matrix.csv",
                 "embeddings_full.csv"]:
    if os.path.exists(filename):
        size = os.path.getsize(filename) / 1024  # в КБ
        print(f"  {filename}: {size:.1f} КБ")

print("\nДля скачивания файлов используйте:")
print("  from google.colab import files")
print("  files.download('имя_файла.csv')")

Сохранение результатов...
✓ news_with_features.csv - основной DataFrame с текстами и признаками
✓ freq_dict_original.csv - частотный словарь исходных слов
✓ freq_dict_lemmas.csv - частотный словарь лемм
✓ bow_matrix.csv - матрица Bag of Words
✓ embeddings_full.csv - полные векторные представления
✓ processing_stats.csv - статистика обработки
✓ README.txt - описание файлов и процесса обработки

ВСЕ ФАЙЛЫ УСПЕШНО СОХРАНЕНЫ!

Размеры файлов:
  news_with_features.csv: 511.7 КБ
  freq_dict_original.csv: 58.2 КБ
  freq_dict_lemmas.csv: 43.5 КБ
  bow_matrix.csv: 62.2 КБ
  embeddings_full.csv: 232.2 КБ

Для скачивания файлов используйте:
  from google.colab import files
  files.download('имя_файла.csv')
